In [24]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd
import numpy as np

In [32]:
def generatePairs(frames):
    framePairs = [] #vector to hold image pairs

    for x in range(len(frames)): #x represents the first frame in the pair
        for y in range(len(frames)-2-x): #y represents second frame in the pair
            pair = [(x+1),(x+y+3),np.stack((frames[x], frames[x+y+2]), axis=-1)] #generate pair
            framePairs.append(pair)
    return framePairs

In [33]:
def translatePair(pair,steps,patient):

    frame1 = pair[0]-1 #get first frame position from pair object
    frame2 = pair[1]-1 #get second frame position from pair object
    JAdd = pair[2].shape[0] #Get size of original frame to keep translation size consistent
    IAdd = pair[2].shape[1] #Get size of original frame to keep translation size consistent

    baseCut = 0.1 #use same base cut as is used in original frame generation
    step = baseCut/steps #set step size to be as large as possible for given number of steps
    pairs = [] #vector to hold newly generated pairs

    #adjust how the frame is cropped from the original data to simulate translation
    for x in range(-steps,steps+1): 
        for y in range(-steps,steps+1):
            lowerI = int((cutLeft+x*step)*256)
            upperI = lowerI+IAdd
            lowerJ = int((cutTop+y*step)*2592)
            upperJ = lowerJ+JAdd

            first = rawData[patient][1][lowerJ:upperJ,lowerI:upperI,frame1]
            second = rawData[patient][1][lowerJ:upperJ,lowerI:upperI,frame2]

            pairs.append(np.stack((first,second), axis=-1)) #append translated pair to pairs vector
    return pairs

In [43]:
#Directory containing the RF scan data
dataFolder = 'C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/'

#Vector to hold all training data sets
rawData = []

#Loading in the matlab file information
rawMatFile = loadmat(dataFolder+'P39-W2-S4')
#Contains just the RF data pulled from the .mat file
P39W2S4 = rawMatFile['rf1']
rawData.append(['P39W2S4',P39W2S4])

#repeat above steps for other RF files
rawMatFile = loadmat(dataFolder+'P39-W4-S6')
P39W4S6 = rawMatFile['rf1']
rawData.append(['P39W4S6',P39W4S6])
rawMatFile = loadmat(dataFolder+'P82-W0-S2')
P82W0S2 = rawMatFile['rf1']
rawData.append(['P82W0S2',P82W0S2])
rawMatFile = loadmat(dataFolder+'P87-W0-S3')
P87W0S3 = rawMatFile['rf1']
rawData.append(['P87W0S3',P87W0S3])
rawMatFile = loadmat(dataFolder+'P87-W2-S4')
P87W2S4 = rawMatFile['rf1']
rawData.append(['P87W2S4',P87W2S4])
rawMatFile = loadmat(dataFolder+'P90-W0-S4')
P90W0S4 = rawMatFile['rf1']
rawData.append(['P90W0S4',P90W0S4])
rawMatFile = loadmat(dataFolder+'P94-W1-S3')
P94W1S3 = rawMatFile['rf1']
rawData.append(['P94W1S3',P94W1S3])

print(rawData[0][1].shape[2])
print(rawData[1][1].shape[2])
print(rawData[2][1].shape[2])
print(rawData[3][1].shape[2])
print(rawData[4][1].shape[2])
print(rawData[5][1].shape[2])
print(rawData[6][1].shape[2])

23
23
19
19
20
19
20


In [35]:
#leave a slice of each edge of the scan from being black
cutTop = 0.1
cutLeft = 0.1
cutBottom = 0.1
cutRight = 0.1

lowerI = int(cutLeft*256)
upperI = int((1-cutRight)*256)
lowerJ = int(cutTop*2592)
upperJ = int((1-cutBottom)*2592)

allFrames = [] #vector to hold individual cropped frames

for y in range(len(rawData)):
    frames = []
    for x in range(rawData[y][1].shape[2]): #range is how many frames there are in the set
        frames.append(rawData[y][1][lowerJ:upperJ,lowerI:upperI,x]) #Take inner subset of frame to allow for translation
    allFrames.append([rawData[y][0],frames])

pairs = []
for x in range(len(allFrames)):
    pairs.append([allFrames[x][0],generatePairs(allFrames[x][1])]) #add generated pairs to list
print(pairs[6][1][0][2].shape)

#PAIRS VARIABLE GUIDE
#First value = Holds which patient set is being referred to
#Second value = Holds either patient set name [0] or generated pairs for set
#Third value = Holds all pairs for patient set
#Fourth value = holds specific pairs for patient set
#pairs[0][1][0][2].shape <- references the shape of the third pair of frames for a certain patients data

(1805, 205, 2)


In [36]:
#Create Vector of stacked pairs with frame number and patient ID for easy labeling
train = []
for x in range(len(pairs)):
    for y in range(len(pairs[x][1])):
        train.append([pairs[x][0],pairs[x][1][y][0],pairs[x][1][y][1],pairs[x][1][y][2]])

print(len(train))
#for x in range(200):
#    print(train[x][0],train[x][1],train[x][2],train[x][3].shape)

1263


In [37]:
#Read in label data
DFP39W2S4 = pd.read_excel('Labels.xlsx', sheet_name='P39-W2-S4', engine='openpyxl')
DFP39W4S6 = pd.read_excel('Labels.xlsx', sheet_name='P39-W4-S6', engine='openpyxl')
DFP82W0S2 = pd.read_excel('Labels.xlsx', sheet_name='P82-W0-S2', engine='openpyxl')
DFP87W0S3 = pd.read_excel('Labels.xlsx', sheet_name='P87-W0-S3', engine='openpyxl')
DFP87W2S4 = pd.read_excel('Labels.xlsx', sheet_name='P87-W2-S4', engine='openpyxl')
DFP90W0S4 = pd.read_excel('Labels.xlsx', sheet_name='P90-W0-S4', engine='openpyxl')
DFP94W1S3 = pd.read_excel('Labels.xlsx', sheet_name='P94-W1-S3', engine='openpyxl')

In [31]:
#create labels for training data
labels = DFP39W2S4['Label axial'].tolist() + DFP39W4S6['Label axial'].tolist() + DFP82W0S2['Label axial'].tolist() + DFP87W0S3['Label axial'].tolist() + DFP87W2S4['Label axial'].tolist() + DFP90W0S4['Label axial'].tolist() + DFP94W1S3['Label axial'].tolist()
print(len(DFP39W2S4['Label axial'].tolist()))
print(len(labels))

210
1470


In [ ]:
import re #for regular expressions to allow sorting the frames

def numberRegex(e):
  #Finding all digits in the filename
  x = re.findall("(\d)", e)

  #if there are two digits in filename
  if(len(x)==2):
      return int(x[0]+""+x[1])
  #if there is one digit in the filename
  else:
      return int(x[0])


In [11]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd

#Testing to see if we can make dataframe straight from the raw mat file dictionary
dataFolder = 'H:/Documents/Github repositories/SE4450-project-code/Preprocessing/P82-W0-S2.mat'
rawMatFile = loadmat(dataFolder)
print(rawMatFile['rf1'].shape[2])


19


In [32]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd

#Lucas directory
#C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Ian directory
#H:/Documents/Github repositories/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Directory containing the RF scan data
dataFolder = 'H:/Documents/Github repositories/SE4450-project-code/Preprocessing/P82-W0-S2.mat'
#vecRFDataNames = os.listdir(os.path.join(dataFolder)) #vector with the names of the images in the above folder
#vecRFDataNames.sort(key=numberRegex) #Sorts the images by frame number

#Loading in the matlab file information
rawMatFile = loadmat(dataFolder)

#Contains just the RF data pulled from the .mat file
#RFData = [[element for element in upperElement] for upperElement in rawMatFile['rf1']]
RFData = rawMatFile['rf1']
print(type(RFData))
print(type(rawMatFile['rf1']))
#Kinda works to leave a slice of each edge of the scan from being black
cutTop = 0.1
cutLeft = 0.1
cutBottom = 0.1
cutRight = 0.1

lowerI = int(cutLeft*256)
upperI = int((1-cutRight)*256)
lowerJ = int(cutTop*2592)
upperJ = int((1-cutBottom)*2592)

#zeroArray = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(256):
    for j in range(2592):
        if j>lowerJ and j<upperJ: #rows to zero out
            if i>lowerI and i<upperI: #cols to zero out
                RFData[j][i][0] = 0 #the 0 in the third [] pertains to the frame to edit (0 being frame 1)
 
# Updates inputted matlab file to contain changes from the for loop above
rawMatFile['rf1'] = RFData

# Saves file while applying changes from the for loop above
# matDic = {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Oct 21 17:26:33 2020','__version__':'1.0', '__globals__':'','rf1':RFData}
# savemat("transformTest.mat", matDic, long_field_names=True)

# Saves the mat file without any changes
savemat("transformTestUnchanged.mat", rawMatFile)

#Converting python dictionary into pandas dataframe for easy manipulation (unsure if needed)
#RFDataframe = pd.DataFrame(rawMatFile['rf1'])

#print(RFDataframe.head(5))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
105


In [ ]:
#This was a long shot to visualize the RF dataframe
import matplotlib.pyplot as plt
import numpy as np

[m,n] = np.shape(RFData[0:10][0:10])

#Colour Map using Matrix
plt.figure()
plt.imshow(RFData, alpha=0.8)
plt.xticks(np.arange(n))
plt.yticks(np.arange(m))
plt.xlabel('Numbers')
plt.ylabel('Value')
plt.title('Color Maps')